[![Open In Colab ](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JDekun/Double-Contrast/blob/main/unet/unet.ipynb?authuser=1) 
[![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/mydekun/U-Net) 
[![Open In Github](https://badgen.net/badge/icon/Open%20MoCo?icon=github&label)](https://github.com/JDekun/Double-Contrast/blob/main/unet/unet.ipynb)

In [1]:
#@title [Colab] 构建Kaggle环境 && 下载数据集

'''
自动化代码，当值 colab 断路
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
'''
# clearInterval()

######### 数据集名字 #########
datasets = [" mydekun/drive"]  #@param {type:"raw"}
import os

# 构建 working input 文件夹和文件
if not os.path.exists('/content/working'):
  !mkdir /content/working
if not os.path.exists('/content/input'):
  !mkdir /content/input

# 导入Kaggle API
os.chdir('/content')
!git clone https://github.com/JDekun/Kaggle.git > /dev/null 2>&1
os.environ['KAGGLE_CONFIG_DIR'] = '/content/Kaggle' #注意kaggle文件夹包含json文件 

# 下载数据集
os.chdir('/content/input')
len_mydekun = len(datasets)
if len_mydekun != 0:
  for i in range(len_mydekun):
    # 下载 kaggle 数据集
    temp = datasets[i]
    name, dataset = temp.split('/')
    dirlist = os.listdir('/content/input')

    if (dataset in dirlist):
      print("您已经下载过'%s'数据集" % dataset)
    else:
      zip = dataset + '.zip'
      !kaggle datasets download -d $temp -p $dataset
      # 解压数据集并删除压缩包
      !unzip $dataset/$zip -d $dataset > /dev/null 2>&1
      !rm -f $dataset/$zip

 63% 18.0M/28.4M [00:00<00:00, 93.7MB/s]
100% 28.4M/28.4M [00:00<00:00, 122MB/s] 


In [2]:
#@title [Kaggle && Colab] 克隆github项目
import os
######### GitHub库 #########
github = "https://github.com/JDekun/deep-learning-for-image-processing.git"  #@param {type:"string"}
## 工作路径
working = github.rsplit('/',1)[-1].rsplit('.',1)[0]

try:
  os.chdir("/kaggle/working/")
except:
  os.chdir("/content/working/")

!git clone $github   # 克隆 github 项目

os.chdir(working)

Cloning into 'deep-learning-for-image-processing'...
remote: Enumerating objects: 6727, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 6727 (delta 34), reused 32 (delta 18), pack-reused 6678
Receiving objects: 100% (6727/6727), 23.61 MiB | 21.32 MiB/s, done.
Resolving deltas: 100% (4204/4204), done.


===============================分割线===============================

In [3]:
os.chdir("pytorch_segmentation/unet")

In [14]:
# run in colab with Tesla T4

!torchrun --nproc_per_node=1 train_multi_GPU.py --data-path ../../../input/drive --b 16 --amp true 2>&1 | tee ./output/output.log 

| distributed init (rank 0): env://
Namespace(amp=True, batch_size=16, data_path='../../../../input/drive', device='cuda', dist_backend='nccl', dist_url='env://', distributed=True, epochs=200, gpu=0, lr=0.01, momentum=0.9, num_classes=1, output_dir='./multi_train', print_freq=1, rank=0, resume='', save_best=True, start_epoch=0, sync_bn=False, test_only=False, weight_decay=0.0001, workers=4, world_size=1)
Creating data loaders
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Creating model
Start training
Epoch: [0]  [0/1]  eta: 0:00:03  lr: 0.010000  loss: 1.3445 (1.3445)  time: 3.1898  

In [1]:
!gpustat -cpu

GPU2                           Tue Oct 18 14:38:07 2022  510.85.02
[0] NVIDIA GeForce GTX 1080 Ti | 72'C,  99 % |  6082 / 11264 MB | yjw:python/36669(5993M)
[1] NVIDIA GeForce GTX 1080 Ti | 69'C,  97 % |  8564 / 11264 MB | yjw:python/36824(8475M)
[2] NVIDIA GeForce GTX 1080 Ti | 76'C,  99 % |  9868 / 11264 MB | yjw:python/38104(9779M)
[3] NVIDIA GeForce GTX 1080 Ti | 74'C, 100 % |  9176 / 11264 MB | byy:python3/20015(9087M)
[4] NVIDIA GeForce GTX 1080 Ti | 63'C,  89 % | 10646 / 11264 MB | byy:python3/20016(10557M)
[5] NVIDIA GeForce GTX 1080 Ti | 78'C,  96 % |  6548 / 11264 MB | zyf:python/6682(6459M)
[6] NVIDIA GeForce GTX 1080 Ti | 72'C,  80 % |  6398 / 11264 MB | zyf:python/3926(6309M)
[7] NVIDIA GeForce GTX 1080 Ti | 70'C,  98 % |  6398 / 11264 MB | zyf:python/28368(6309M)
[8] NVIDIA GeForce GTX 1080 Ti | 28'C,   0 % |    87 / 11264 MB |


In [ ]:
# run in 318 with 1080Ti

!CUDA_VISIBLE_DEVICES=3 torchrun --nproc_per_node=1 train_multi_GPU.py --data-path ../../../input/drive --b 8 --amp true 2>&1 | tee ./output/output.log 